In [1]:
# ------------------ التحميلات الأساسية ------------------
import os
import re
import html
import joblib
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# تحميل الموارد اللازمة لـ NLTK (مرة واحدة فقط)
import nltk
nltk.download('stopwords')

# ------------------ تعريف دالة التنظيف ------------------
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def advanced_preprocess(text):
    text = html.unescape(text)
    text = ''.join(c for c in text if c.isprintable())
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words and len(w) > 2]
    return ' '.join(words)

# ------------------ 1. تحديد المسارات ------------------
print("--- 1. تحديد مسارات الملفات ---")
doc_ids_path = r"C:\Users\Mohammad Mihdi\Desktop\Projects\IR\data\msmarco_train\index\TFIDF\doc_ids_msmarco_train.joblib"
tfidf_matrix_path = r"C:\Users\Mohammad Mihdi\Desktop\Projects\IR\data\msmarco_train\index\TFIDF\tfidf_matrix_msmarco_train.joblib"
tfidf_vectorizer_path = r"C:\Users\Mohammad Mihdi\Desktop\Projects\IR\data\msmarco_train\index\TFIDF\tfidf_vectorizer_msmarco_train.joblib"

output_inverted_index_path = r"C:\Users\Mohammad Mihdi\Desktop\Projects\IR\data\msmarco_train\index\TFIDF\tfidf_inverted_index.joblib"

output_dir = os.path.dirname(output_inverted_index_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"تم إنشاء مجلد الإخراج: {output_dir}")
else:
    print(f"مجلد الإخراج موجود: {output_dir}")

# ------------------ 2. تحميل الملفات ------------------
print("\n--- 2. جاري تحميل الملفات ---")
try:
    doc_ids = joblib.load(doc_ids_path)
    tfidf_matrix = joblib.load(tfidf_matrix_path)
    tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)
    print("تم تحميل الملفات بنجاح.")
    print(f"عدد المستندات: {len(doc_ids)}")
    print(f"أبعاد مصفوفة TF-IDF: {tfidf_matrix.shape}")
except FileNotFoundError as e:
    print(f"خطأ: لم يتم العثور على أحد الملفات. يرجى التحقق من المسارات.")
    print(f"المسار المفقود: {e.filename}")
    exit()
except Exception as e:
    print(f"حدث خطأ أثناء تحميل الملفات: {e}")
    exit()

# ------------------ 3. بناء الفهرس المعكوس ------------------
print("\n--- 3. جاري بناء الفهرس المعكوس ---")

feature_names = tfidf_vectorizer.get_feature_names_out()
inverted_index = {}

tfidf_matrix_coo = tfidf_matrix.tocoo()
print("بدء عملية التكرار على مصفوفة TF-IDF (قد تستغرق بعض الوقت)...")

for doc_idx, term_idx, tfidf_score in tqdm(zip(tfidf_matrix_coo.row, tfidf_matrix_coo.col, tfidf_matrix_coo.data),
                                           total=len(tfidf_matrix_coo.data),
                                           desc="بناء الفهرس المعكوس"):
    term = feature_names[term_idx]
    doc_id = doc_ids[doc_idx]
    if term not in inverted_index:
        inverted_index[term] = []
    inverted_index[term].append((doc_id, float(tfidf_score)))

print("جاري فرز الوثائق لكل مصطلح حسب درجة TF-IDF...")
for term in inverted_index:
    inverted_index[term].sort(key=lambda x: x[1], reverse=True)

print("تم بناء الفهرس المعكوس بنجاح.")
print(f"عدد الكلمات الفريدة في الفهرس المعكوس: {len(inverted_index)}")

# ------------------ 4. إعداد البيانات للحفظ ------------------
print("\n--- 4. إعداد البيانات للحفظ ---")
inverted_index_data = {
    "inverted_index": inverted_index,
    "num_documents": len(doc_ids),
    "num_terms": len(inverted_index),
    "vocabulary_size": len(feature_names),
    "vectorizer_vocabulary": dict(tfidf_vectorizer.vocabulary_)
}

# ------------------ 5. حفظ الفهرس ------------------
print("\n--- 5. جاري حفظ الفهرس المعكوس ---")
try:
    joblib.dump(inverted_index_data, output_inverted_index_path)
    print(f"✅ تم حفظ الفهرس المعكوس بنجاح في: {output_inverted_index_path}")
except Exception as e:
    print(f"❌ حدث خطأ أثناء حفظ الفهرس المعكوس: {e}")

# ------------------ 6. التحقق من الحفظ ------------------
print("\n--- 6. التحقق من الفهرس المحفوظ ---")
try:
    loaded_data = joblib.load(output_inverted_index_path)
    loaded_index = loaded_data["inverted_index"]
    print("✅ تم تحميل الفهرس للتحقق.")
    print(f"عدد الكلمات في الفهرس: {len(loaded_index)}")

    example_term = next(iter(loaded_index))
    print(f"مثال على إدخال في الفهرس المعكوس ('{example_term}'): {loaded_index[example_term][:5]}")
except Exception as e:
    print(f"❌ حدث خطأ أثناء التحقق: {e}")

print("\n--- ✅ انتهى تنفيذ الكود بنجاح ---")


[nltk_data] Downloading package stopwords to C:\Users\Mohammad
[nltk_data]     Mihdi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


--- 1. تحديد مسارات الملفات ---
مجلد الإخراج موجود: C:\Users\Mohammad Mihdi\Desktop\Projects\IR\data\msmarco_train\index\TFIDF

--- 2. جاري تحميل الملفات ---
تم تحميل الملفات بنجاح.
عدد المستندات: 500000
أبعاد مصفوفة TF-IDF: (500000, 86423)

--- 3. جاري بناء الفهرس المعكوس ---
بدء عملية التكرار على مصفوفة TF-IDF (قد تستغرق بعض الوقت)...


بناء الفهرس المعكوس:   0%|          | 0/11542832 [00:00<?, ?it/s]

جاري فرز الوثائق لكل مصطلح حسب درجة TF-IDF...
تم بناء الفهرس المعكوس بنجاح.
عدد الكلمات الفريدة في الفهرس المعكوس: 86423

--- 4. إعداد البيانات للحفظ ---

--- 5. جاري حفظ الفهرس المعكوس ---
✅ تم حفظ الفهرس المعكوس بنجاح في: C:\Users\Mohammad Mihdi\Desktop\Projects\IR\data\msmarco_train\index\TFIDF\tfidf_inverted_index.joblib

--- 6. التحقق من الفهرس المحفوظ ---
✅ تم تحميل الفهرس للتحقق.
عدد الكلمات في الفهرس: 86423
مثال على إدخال في الفهرس المعكوس ('presenc'): [('409792', 0.4636915791740488), ('580660', 0.4439570255952832), ('774505', 0.4316726585819526), ('569503', 0.41680316950662577), ('585454', 0.3903875694978039)]

--- ✅ انتهى تنفيذ الكود بنجاح ---


In [1]:
# ------------------ التحميلات الأساسية ------------------
import os
import re
import html
import joblib
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# تحميل الموارد اللازمة لـ NLTK (مرة واحدة فقط)
import nltk
nltk.download('stopwords')

# ------------------ تعريف دالة التنظيف ------------------
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def advanced_preprocess(text):
    text = html.unescape(text)
    text = ''.join(c for c in text if c.isprintable())
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words and len(w) > 2]
    return ' '.join(words)

# ------------------ 1. تحديد المسارات ------------------
print("--- 1. تحديد مسارات الملفات ---")
doc_ids_path = r"..\data\antique_train\index\TFIDF\doc_ids_antique_train.joblib"
tfidf_matrix_path = r"..\data\antique_train\index\TFIDF\tfidf_matrix_antique_train.joblib"
tfidf_vectorizer_path = r"..\data\antique_train\index\TFIDF\tfidf_vectorizer_antique_train.joblib"

output_inverted_index_path = r"..\data\antique_train\index\TFIDF\tfidf_inverted_index.joblib"

output_dir = os.path.dirname(output_inverted_index_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"تم إنشاء مجلد الإخراج: {output_dir}")
else:
    print(f"مجلد الإخراج موجود: {output_dir}")

# ------------------ 2. تحميل الملفات ------------------
print("\n--- 2. جاري تحميل الملفات ---")
try:
    doc_ids = joblib.load(doc_ids_path)
    tfidf_matrix = joblib.load(tfidf_matrix_path)
    tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)
    print("تم تحميل الملفات بنجاح.")
    print(f"عدد المستندات: {len(doc_ids)}")
    print(f"أبعاد مصفوفة TF-IDF: {tfidf_matrix.shape}")
except FileNotFoundError as e:
    print(f"خطأ: لم يتم العثور على أحد الملفات. يرجى التحقق من المسارات.")
    print(f"المسار المفقود: {e.filename}")
    exit()
except Exception as e:
    print(f"حدث خطأ أثناء تحميل الملفات: {e}")
    exit()

# ------------------ 3. بناء الفهرس المعكوس ------------------
print("\n--- 3. جاري بناء الفهرس المعكوس ---")

feature_names = tfidf_vectorizer.get_feature_names_out()
inverted_index = {}

tfidf_matrix_coo = tfidf_matrix.tocoo()
print("بدء عملية التكرار على مصفوفة TF-IDF (قد تستغرق بعض الوقت)...")

for doc_idx, term_idx, tfidf_score in tqdm(zip(tfidf_matrix_coo.row, tfidf_matrix_coo.col, tfidf_matrix_coo.data),
                                           total=len(tfidf_matrix_coo.data),
                                           desc="بناء الفهرس المعكوس"):
    term = feature_names[term_idx]
    doc_id = doc_ids[doc_idx]
    if term not in inverted_index:
        inverted_index[term] = []
    inverted_index[term].append((doc_id, float(tfidf_score)))

print("جاري فرز الوثائق لكل مصطلح حسب درجة TF-IDF...")
for term in inverted_index:
    inverted_index[term].sort(key=lambda x: x[1], reverse=True)

print("تم بناء الفهرس المعكوس بنجاح.")
print(f"عدد الكلمات الفريدة في الفهرس المعكوس: {len(inverted_index)}")

# ------------------ 4. إعداد البيانات للحفظ ------------------
print("\n--- 4. إعداد البيانات للحفظ ---")
inverted_index_data = {
    "inverted_index": inverted_index,
    "num_documents": len(doc_ids),
    "num_terms": len(inverted_index),
    "vocabulary_size": len(feature_names),
    "vectorizer_vocabulary": dict(tfidf_vectorizer.vocabulary_)
}

# ------------------ 5. حفظ الفهرس ------------------
print("\n--- 5. جاري حفظ الفهرس المعكوس ---")
try:
    joblib.dump(inverted_index_data, output_inverted_index_path)
    print(f"✅ تم حفظ الفهرس المعكوس بنجاح في: {output_inverted_index_path}")
except Exception as e:
    print(f"❌ حدث خطأ أثناء حفظ الفهرس المعكوس: {e}")

# ------------------ 6. التحقق من الحفظ ------------------
print("\n--- 6. التحقق من الفهرس المحفوظ ---")
try:
    loaded_data = joblib.load(output_inverted_index_path)
    loaded_index = loaded_data["inverted_index"]
    print("✅ تم تحميل الفهرس للتحقق.")
    print(f"عدد الكلمات في الفهرس: {len(loaded_index)}")

    example_term = next(iter(loaded_index))
    print(f"مثال على إدخال في الفهرس المعكوس ('{example_term}'): {loaded_index[example_term][:5]}")
except Exception as e:
    print(f"❌ حدث خطأ أثناء التحقق: {e}")

print("\n--- ✅ انتهى تنفيذ الكود بنجاح ---")


[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


--- 1. تحديد مسارات الملفات ---
مجلد الإخراج موجود: ..\data\antique_train\index\TFIDF

--- 2. جاري تحميل الملفات ---
تم تحميل الملفات بنجاح.
عدد المستندات: 374507
أبعاد مصفوفة TF-IDF: (374507, 47612)

--- 3. جاري بناء الفهرس المعكوس ---
بدء عملية التكرار على مصفوفة TF-IDF (قد تستغرق بعض الوقت)...


بناء الفهرس المعكوس:   0%|          | 0/5812829 [00:00<?, ?it/s]

جاري فرز الوثائق لكل مصطلح حسب درجة TF-IDF...
تم بناء الفهرس المعكوس بنجاح.
عدد الكلمات الفريدة في الفهرس المعكوس: 47612

--- 4. إعداد البيانات للحفظ ---

--- 5. جاري حفظ الفهرس المعكوس ---
✅ تم حفظ الفهرس المعكوس بنجاح في: ..\data\antique_train\index\TFIDF\tfidf_inverted_index.joblib

--- 6. التحقق من الفهرس المحفوظ ---
✅ تم تحميل الفهرس للتحقق.
عدد الكلمات في الفهرس: 47612
مثال على إدخال في الفهرس المعكوس ('small'): [('3021359_3', 1.0), ('3890001_1', 1.0), ('492280_2', 1.0), ('1385576_10', 0.7901621905642784), ('3455154_7', 0.7901621905642784)]

--- ✅ انتهى تنفيذ الكود بنجاح ---


In [ ]:
# ------------------ التحميلات الأساسية ------------------
import os
import re
import html
import joblib
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

import sys
sys.path.append("..")

from services.documents_service import preprocess

# ------------------ 1. تحديد المسارات ------------------
print("--- 1. تحديد مسارات الملفات ---")
doc_ids_path = r"..\data\beir_quora_test\index\TFIDF\doc_ids_beir_quora_test.joblib"
tfidf_matrix_path = r"..\data\beir_quora_test\index\TFIDF\tfidf_matrix_beir_quora_test.joblib"
tfidf_vectorizer_path = r"..\data\beir_quora_test\index\TFIDF\tfidf_vectorizer_beir_quora_test.joblib"

output_inverted_index_path = r"..\data\beir_quora_test\index\TFIDF\tfidf_inverted_index.joblib"

output_dir = os.path.dirname(output_inverted_index_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"تم إنشاء مجلد الإخراج: {output_dir}")
else:
    print(f"مجلد الإخراج موجود: {output_dir}")

# ------------------ 2. تحميل الملفات ------------------
print("\n--- 2. جاري تحميل الملفات ---")
try:
    doc_ids = joblib.load(doc_ids_path)
    tfidf_matrix = joblib.load(tfidf_matrix_path)
    tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)
    print("تم تحميل الملفات بنجاح.")
    print(f"عدد المستندات: {len(doc_ids)}")
    print(f"أبعاد مصفوفة TF-IDF: {tfidf_matrix.shape}")
except FileNotFoundError as e:
    print(f"خطأ: لم يتم العثور على أحد الملفات. يرجى التحقق من المسارات.")
    print(f"المسار المفقود: {e.filename}")
    exit()
except Exception as e:
    print(f"حدث خطأ أثناء تحميل الملفات: {e}")
    exit()

# ------------------ 3. بناء الفهرس المعكوس ------------------
print("\n--- 3. جاري بناء الفهرس المعكوس ---")

feature_names = tfidf_vectorizer.get_feature_names_out()
inverted_index = {}

tfidf_matrix_coo = tfidf_matrix.tocoo()
print("بدء عملية التكرار على مصفوفة TF-IDF (قد تستغرق بعض الوقت)...")

for doc_idx, term_idx, tfidf_score in tqdm(zip(tfidf_matrix_coo.row, tfidf_matrix_coo.col, tfidf_matrix_coo.data),
                                           total=len(tfidf_matrix_coo.data),
                                           desc="بناء الفهرس المعكوس"):
    term = feature_names[term_idx]
    doc_id = doc_ids[doc_idx]
    if term not in inverted_index:
        inverted_index[term] = []
    inverted_index[term].append((doc_id, float(tfidf_score)))

print("جاري فرز الوثائق لكل مصطلح حسب درجة TF-IDF...")
for term in inverted_index:
    inverted_index[term].sort(key=lambda x: x[1], reverse=True)

print("تم بناء الفهرس المعكوس بنجاح.")
print(f"عدد الكلمات الفريدة في الفهرس المعكوس: {len(inverted_index)}")

# ------------------ 4. إعداد البيانات للحفظ ------------------
print("\n--- 4. إعداد البيانات للحفظ ---")
inverted_index_data = {
    "inverted_index": inverted_index,
    "num_documents": len(doc_ids),
    "num_terms": len(inverted_index),
    "vocabulary_size": len(feature_names),
    "vectorizer_vocabulary": dict(tfidf_vectorizer.vocabulary_)
}

# ------------------ 5. حفظ الفهرس ------------------
print("\n--- 5. جاري حفظ الفهرس المعكوس ---")
try:
    joblib.dump(inverted_index_data, output_inverted_index_path)
    print(f"✅ تم حفظ الفهرس المعكوس بنجاح في: {output_inverted_index_path}")
except Exception as e:
    print(f"❌ حدث خطأ أثناء حفظ الفهرس المعكوس: {e}")

# ------------------ 6. التحقق من الحفظ ------------------
print("\n--- 6. التحقق من الفهرس المحفوظ ---")
try:
    loaded_data = joblib.load(output_inverted_index_path)
    loaded_index = loaded_data["inverted_index"]
    print("✅ تم تحميل الفهرس للتحقق.")
    print(f"عدد الكلمات في الفهرس: {len(loaded_index)}")

    example_term = next(iter(loaded_index))
    print(f"مثال على إدخال في الفهرس المعكوس ('{example_term}'): {loaded_index[example_term][:5]}")
except Exception as e:
    print(f"❌ حدث خطأ أثناء التحقق: {e}")

print("\n--- ✅ انتهى تنفيذ الكود بنجاح ---")
